# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



## <font color='#565F76'>Работу выполняли</font>
* <font color='#565F76'>__Лукьянов Андрей__</font>
* <font color='#565F76'>__Лягушкина Дарья__</font>
* <font color='#565F76'>__Родыгина Анастасия__</font>

### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

<font color='#565F76'>__Для выполнения этого задания мы использовали scrapy - фреймворк, позволяющий парсить (или, проще говоря, "доставать") данные с сайтов. С его помощью мы смогли создать паука и в результате получить json-файл ("data.json") с прогнозом погоды на ближайшие 10 дней. Нам также пришлось обратиться к регулярным выражениям, чтобы преобразовать данные в более удобный формат.__</font>

## <font color='#565F76'>Наш паук, выполняющий задание, находится в том же архиве, что и этот ноутбук.</font>

### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

<font color='#565F76'>__В этом задании нам понадобилась библиотека pymorphy, которая умеет определять и менять формы слова в соответствии с родом, числом и падежом. Это помогло нам создать шаблоны на правильном русском языке.__</font>

In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

 <font color='#565F76'>__Шаблон 1: В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2.__</font>

 <font color='#565F76'>__Описание каждой функции можно найти в Docstring.__</font>

In [2]:
def degree(day, gap_degree, comparative_day):
    """
    Генерирует текст прогноза изменения температуры.
    
    day  -  type: string, lower case. День, для которого делается прогноз.
    gap_degree  -  type: int. Изменение температуры.
    comparative_day  -  type: string, lower case. День, по сравнению с которым делается прогноз.
    
    Return: Шаблон 1: В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    
    """
    if gap_degree >= 0:
        weather = "потеплеет на"
    elif gap_degree <0 :
        weather = "похолодает на"
        gap_degree = gap_degree*(-1)
        
    #Вторник - единственный день, с которым нужно использовать предлог "Во", а не "в"
    if day == "вторник":
        predlog = 'Во'
    else:
        predlog = 'В'
    
    #Вторник,среда - единственные дни, с которыми нужно использовать предлог "со", а не "с", насчет среды слегка сомневаюсь
    if comparative_day == "вторник" or comparative_day == "среда":
        predlog2 = 'со'
    else:
        predlog2 = 'с'
    day1 = morph.parse(day)[0]
    comparative_day1 = morph.parse(comparative_day)[0]
    parsed_word = morph.parse('градусы')[0]
    prognoz = ' '.join([predlog, day1.inflect({'accs'}).word, 'в Москве', weather, str(gap_degree),
                        parsed_word.make_agree_with_number(gap_degree).word, 'по сравнению', predlog2,
                        comparative_day1.inflect({'ablt'}).word + '.'])
    return prognoz

In [12]:
degree('четверг', 8, 'среда')

'В четверг в Москве потеплеет на 8 градусов по сравнению со средой.'

 <font color='#565F76'>__Шаблон 2: Скорость ветра изменится на X единиц в день1 по сравнению с день2.__</font>

In [13]:
def wind(day, gap_wind, comparative_day):
    """
    Генерирует текст прогноза изменения скорости ветра.
    
    day  -  type: string, lower case. День, для которого делается прогноз.
    gap_wind  -  type: int. Изменение скорости ветра.
    comparative_day  -  type: string, lower case. День, по сравнению с которым делается прогноз.
    
    Return: Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    
    """
    #Вторник - единственный день, с которым нужно использовать предлог "Во", а не "в"
    if day == "вторник":
        predlog = 'во'
    else:
        predlog = 'в'
    #Вторник,среда - единственные дни, с которыми нужно использовать предлог "со", а не "с"
    if comparative_day == "вторник" or comparative_day == "среда":
        predlog2 = 'со'
    else:
        predlog2 = 'с'
    day1 = morph.parse(day)[0]
    comparative_day1 = morph.parse(comparative_day)[0]
    parsed_word = morph.parse('единица')[0]
    prognoz = ' '.join(['Скорость ветра изменится на', str(gap_wind),
                        parsed_word.make_agree_with_number(gap_wind).word, predlog, day1.inflect({'accs'}).word,
                        'по сравнению', predlog2, comparative_day1.inflect({'ablt'}).word + '.'])
    return prognoz

In [14]:
wind('вторник', 5, 'среда')

'Скорость ветра изменится на 5 единиц во вторник по сравнению со средой.'

 <font color='#565F76'>__Шаблон 3: Уровень осадков повысится / понизится на X единиц за Y дней.__ </font>

In [15]:
def precipitation(gap_rain, n_days):
    """
    Генерирует текст прогноза изменения уровня осадков за n дней.
    
    gap_rain  -  type: float. Изменение уровня осадков.
    n_days  -  type: int. Количество дней.
    
    Return: Уровень осадков повысится / понизится на X единиц за Y дней.
    
    """
    
    if gap_rain >= 0:
        rain = "повысится на"
    elif gap_rain < 0:
        rain = "понизится на"
        gap_rain = gap_rain*(-1)
    parsed_word = morph.parse('единица')[0]
    parsed_word_2 = morph.parse('день')[0]
    prognoz = ' '.join(['Уровень осадков', rain, str(gap_rain), parsed_word.make_agree_with_number(gap_rain).word,
                        'за', str(n_days), parsed_word_2.make_agree_with_number(n_days).word + '.'])
    return prognoz

In [16]:
precipitation(-10, 2)

'Уровень осадков понизится на 10 единиц за 2 дня.'

In [17]:
with open(r"data.json", 'r',encoding = "utf-8") as infile:
    data = infile.read()
    print(data)

[
{
  "Dates": [
    "13.02 (вт)",
    "14.02 (ср)",
    "15.02 (чт)",
    "16.02 (пт)",
    "17.02 (сб)",
    "18.02 (вс)",
    "19.02 (пн)",
    "20.02 (вт)",
    "21.02 (ср)",
    "22.02 (чт)"
  ],
  "Max temperature": [
    "−6",
    "−9",
    "−10",
    "−5",
    "−5",
    "−3",
    "−1",
    "−4",
    "−5",
    "−7"
  ],
  "Min temperature": [
    "−13",
    "−14",
    "−18",
    "−9",
    "−7",
    "−6",
    "−6",
    "−7",
    "−8",
    "−12"
  ],
  "Wind Speed": [
    "6",
    "8",
    "6",
    "8",
    "9",
    "7",
    "11",
    "9",
    "5",
    "5"
  ],
  "Precipitation": [
    "0",
    "0",
    "0,1",
    "0,8",
    "2,8",
    "1,2",
    "0,4",
    "0",
    "0,6",
    "0,4"
  ]
}
]


In [18]:
import pandas as pd

df = pd.read_json('data.json')
df

,Dates,Max temperature,Min temperature,Precipitation,Wind Speed
0,"[13.02 (вт), 14.02 (ср), 15.02 (чт), 16.02 (пт...","[−6, −9, −10, −5, −5, −3, −1, −4, −5, −7]","[−13, −14, −18, −9, −7, −6, −6, −7, −8, −12]","[0, 0, 0,1, 0,8, 2,8, 1,2, 0,4, 0, 0,6, 0,4]","[6, 8, 6, 8, 9, 7, 11, 9, 5, 5]"


In [19]:
#Превращаем это в нормальный DataFrame, но выглядит как-то очень сложно
Dates = pd.DataFrame(df['Dates'][0])
Max_temp = pd.DataFrame(df['Max temperature'][0])
Min_temp = pd.DataFrame(df['Min temperature'][0])
Precip = pd.DataFrame(df['Precipitation'][0])
Wind_Speed = pd.DataFrame(df['Wind Speed'][0])
table = pd.concat((Max_temp,Min_temp,Precip,Wind_Speed), axis=1)
table.columns = ["Max temperature", "Min temperature","Precipitation","Wind Speed"]
table.head()

,Max temperature,Min temperature,Precipitation,Wind Speed
0,−6,−13,0,6
1,−9,−14,0,8
2,−10,−18,"0,1",6
3,−5,−9,"0,8",8
4,−5,−7,"2,8",9


In [20]:
Dates.columns = ['Date']
Dates.head()

,Date
0,13.02 (вт)
1,14.02 (ср)
2,15.02 (чт)
3,16.02 (пт)
4,17.02 (сб)


In [21]:
table = table.set_index(Dates.Date.values)
table

,Max temperature,Min temperature,Precipitation,Wind Speed
13.02 (вт),−6,−13,0,6
14.02 (ср),−9,−14,0,8
15.02 (чт),−10,−18,"0,1",6
16.02 (пт),−5,−9,"0,8",8
17.02 (сб),−5,−7,"2,8",9
18.02 (вс),−3,−6,"1,2",7
19.02 (пн),−1,−6,"0,4",11
20.02 (вт),−4,−7,0,9
21.02 (ср),−5,−8,"0,6",5
22.02 (чт),−7,−12,"0,4",5


##  <font color='#565F76'>Вуаля! </font>

In [22]:
table.T

,13.02 (вт),14.02 (ср),15.02 (чт),16.02 (пт),17.02 (сб),18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт)
Max temperature,−6,−9,−10,−5,−5,−3,−1,−4,−5,−7
Min temperature,−13,−14,−18,−9,−7,−6,−6,−7,−8,−12
Precipitation,0,0,"0,1","0,8","2,8","1,2","0,4",0,"0,6","0,4"
Wind Speed,6,8,6,8,9,7,11,9,5,5


In [23]:
import re

In [24]:
Dates['Date'][1]

'14.02 (ср)'

In [25]:
result = re.search('ср|вт', Dates['Date'] [0])
days_of_week = {'пн': "понедельник",
                'вт': "вторник",
                'ср': "среда",
                'чт': "четверг",
                'пт': "пятница",
                'сб': "суббота",
                'вс': "воскресенье"}
days_of_week[result.group(0)]

'вторник'

In [26]:
days_of_week['ср']

'среда'

 <font color='#565F76'>__Пока начинаем наши прогнозы на день позже с 14 числа__</font>

In [27]:
i = 1
while i < len(table):
    print(Dates['Date'][i])
    result1 = re.search('пн|вт|ср|чт|пт|сб|вс', Dates['Date'][i])
    day = days_of_week[result1.group(0)]
    degree1 = re.sub('−', '-', table['Min temperature'][i])
    degree2 = re.sub('−', '-', table['Min temperature'][i-1])
    gap_degree = int(degree1) - int(degree2)
    result2 = re.search('пн|вт|ср|чт|пт|сб|вс', Dates['Date'][i-1])
    comparative_day = days_of_week[result2.group(0)]
    
    wind1 = re.sub('−', '-', table['Wind Speed'][i])
    wind2 = re.sub('−', '-', table['Wind Speed'][i-1])
    gap_wind = int(wind1) - int(wind2)
    
    rain1 = re.sub(',', '.', table['Precipitation'][0])
    rain2 = re.sub(',', '.', table['Precipitation'][i])
    gap_rain = float(rain2) - float(rain1)
    n_days = i
    
    i += 1
    print(degree(day, gap_degree, comparative_day))
    print(wind(day, gap_wind, comparative_day))
    print(precipitation(gap_rain, n_days))

14.02 (ср)
В среду в Москве похолодает на 1 градус по сравнению со вторником.
Скорость ветра изменится на 2 единицы в среду по сравнению со вторником.
Уровень осадков повысится на 0.0 единиц за 1 день.
15.02 (чт)
В четверг в Москве похолодает на 4 градуса по сравнению со средой.
Скорость ветра изменится на -2 единиц в четверг по сравнению со средой.
Уровень осадков повысится на 0.1 единиц за 2 дня.
16.02 (пт)
В пятницу в Москве потеплеет на 9 градусов по сравнению с четвергом.
Скорость ветра изменится на 2 единицы в пятницу по сравнению с четвергом.
Уровень осадков повысится на 0.8 единиц за 3 дня.
17.02 (сб)
В субботу в Москве потеплеет на 2 градуса по сравнению с пятницей.
Скорость ветра изменится на 1 единица в субботу по сравнению с пятницей.
Уровень осадков повысится на 2.8 единицы за 4 дня.
18.02 (вс)
В воскресенье в Москве потеплеет на 1 градус по сравнению с субботой.
Скорость ветра изменится на -2 единиц в воскресенье по сравнению с субботой.
Уровень осадков повысится на 1.2 е

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

#### Текст по шаблонам
* <font color='#565F76'>__Автоматический ответ на сообщения__</font>
* <font color='#565F76'>__Создание ботов__</font>
* <font color='#565F76'>__Проверка введенных пользователем данных (Пример: ИМЯ, ты согласен/согласна, что данные введены верно?) __</font>
* <font color='#565F76'>__Генерирование отзывов__</font>

#### Текст об изменении числовых показателей по шаблонам
* <font color='#565F76'>__Уведомления в приложениях (Пример: у Вас N непрочитанных сообщений от ИМЯ, Вы потратили N бонусов/бонуса)__</font>
* <font color='#565F76'>__Описание товара на сайте (Пример: на складе осталось N столов/стола)__</font>
* <font color='#565F76'>__Валютные показатели (Пример: рубль вырос на N единиц)__</font>
* <font color='#565F76'>__Персональная статистика (Пример: за последний/последнюю ПЕРИОД_ВРЕМЕНИ Вашу страницу посетило на N пользователей больше/меньше, чем в/на прошлом/прошлой ПЕРИОД_ВРЕМЕНИ)__</font>

* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

####  Как можно разнообразить эти шаблоны
* <font color='#565F76'>__Очевидно, нужно сделать структуру более подвижной.__</font>
* <font color='#565F76'>__Включить менее типичные конструкции предложений.__</font>
* <font color='#565F76'>__Менять последовательность слов.__</font>
* <font color='#565F76'>__Добавлять поясняющие предложения. Например: сегодня возьмите с собой зонтик потому что ожидается дождь.__</font>

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 17.02.2018.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 1: Анна Лапидус (email: anyalapidus@list.ru, telegram: @alpids).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт:

ИАД-1 : VNYQlGd

ИАД-2 : wDEa8ge

ИАД-3 : SviK5dd

ИАД-4 : dzdD8YG


 У вас появится курс “ВШЭ > Прикладные задачи анализа данных” в разделе “Посещает курсы”.
* Перейдите по ссылке “ВШЭ > Прикладные задачи анализа данных (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/281